# Frame Extraction

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path

def get_video_paths(folder_path, extensions=None, recursive=True):
    """
    Get all video file paths from a folder.

    Args:
        folder_path (str): Path to the folder containing videos.
        extensions (list, optional): List of video extensions to look for.
                                     Defaults to common video formats.
        recursive (bool, optional): Whether to search subfolders recursively.
                                    Defaults to True.

    Returns:
        list: List of absolute paths to video files.
    """
    if extensions is None:
        extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.webm', '.wmv']

    video_paths = []

    if recursive:
        for root, _, files in os.walk(folder_path):
            for file in files:
                if any(file.lower().endswith(ext) for ext in extensions):
                    video_paths.append(os.path.join(root, file))
    else:
        for file in os.listdir(folder_path):
            if any(file.lower().endswith(ext) for ext in extensions):
                video_paths.append(os.path.join(folder_path, file))

    return video_paths

def calculate_adaptive_interval(duration, motion_threshold=0.5):
    """
    Calculate frame interval considering both duration and expected motion.

    Args:
        duration (float): Video duration in seconds
        motion_threshold (float): Expected motion level (0-1)

    Returns:
        tuple: (base_interval, motion_check_interval)
    """
    # base interval based on duration
    if duration < 5:
        base = duration / 15  # Ensure at least 15 frames for very short videos
    elif duration < 10:
        base = 0.8
    elif duration < 30:
        base = 1.0
    elif duration < 60:
        base = 1.5
    else:
        base = 0.7

    # lower threshold = more frames
    motion_factor = 1.5 - motion_threshold  # 2x faster check motion
    return (base * motion_factor, base * motion_factor * 0.5)

def extract_diverse_frames(video_paths, base_output_dir="extracted_frames", motion_threshold=0.3):
    """
    Extract frames with emphasis on diversity and key moments.

    Args:
        video_paths (list): List of video file paths
        base_output_dir (str): Output directory for extracted frames
        motion_threshold (float): Sensitivity to motion (0-1, lower=more sensitive)
    """
    Path(base_output_dir).mkdir(parents=True, exist_ok=True)

    for video_path in video_paths:
        video_name = Path(video_path).stem
        output_folder = os.path.join(base_output_dir)
        os.makedirs(output_folder, exist_ok=True)

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Warning: Could not open video {video_path}")
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps

        base_interval, motion_interval = calculate_adaptive_interval(duration, motion_threshold)
        frame_interval = max(1, int(fps * base_interval))
        motion_check_interval = max(1, int(fps * motion_interval))

        print(f"\nProcessing {video_name} ({duration:.1f}s, {total_frames} frames)")
        print(f"Base interval: {base_interval:.2f}s (~{frame_interval} frames)")
        print(f"Motion check every: {motion_interval:.2f}s")

        # always capture first and last frames!!!
        selected_frames = {0, total_frames-1}

        # adding key frames at scene changes
        success, prev_frame = cap.read()
        if not success:
            continue

        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        prev_gray = cv2.GaussianBlur(prev_gray, (21, 21), 0)
        frame_diffs = []

        for frame_num in range(1, total_frames):
            ret, frame = cap.read()
            if not ret:
                break

            # checkking for motion periodically
            if frame_num % motion_check_interval == 0:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (21, 21), 0)
                delta = cv2.absdiff(gray, prev_gray)
                thresh = cv2.threshold(delta, 25, 255, cv2.THRESH_BINARY)[1]
                changed_pixels = cv2.countNonZero(thresh)
                change_percent = changed_pixels / (gray.shape[0] * gray.shape[1])
                frame_diffs.append((frame_num, change_percent))

                if change_percent > (motion_threshold * 0.05):  # scale threshold
                    selected_frames.add(frame_num)
                    prev_gray = gray

            # add frames at regular intervals
            if frame_num % frame_interval == 0:
                selected_frames.add(frame_num)

        # add most dynamic frames if we didnt get enough variation
        if len(selected_frames) < min(30, total_frames) and frame_diffs:
            frame_diffs.sort(key=lambda x: x[1], reverse=True)
            needed = min(30, total_frames) - len(selected_frames)
            for i in range(min(needed, len(frame_diffs))):
                selected_frames.add(frame_diffs[i][0])

        # ensure minimum frame count
        min_frames = min(30, total_frames)
        while len(selected_frames) < min_frames:
            extra_frame = np.random.randint(0, total_frames)
            selected_frames.add(extra_frame)

        # saving selected frames
        saved_count = 0
        for frame_num in sorted(selected_frames):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            success, frame = cap.read()
            if success:
                frame_path = os.path.join(output_folder,
                                       f"{video_name}_frame_{frame_num:06d}.jpg")
                cv2.imwrite(frame_path, frame)
                saved_count += 1

        cap.release()
        print(f"Saved {saved_count} diverse frames to {output_folder}")
        print(f"Frame selection diversity: {len(selected_frames)/min_frames:.1%} of target")


VIDEO_DIR = "/content/drive/MyDrive/yolov11_test_cv/data/raw_video"
OUTPUT_DIR = "/content/drive/MyDrive/yolov11_test_cv/data/extracted_frames"
MOTION_THRESHOLD = 0.3  # lower = more sensitive to motion

if __name__ == "__main__":
    print(f"Searching for videos in {VIDEO_DIR}...")
    video_paths = get_video_paths(VIDEO_DIR)

    if not video_paths:
        print("No videos found in the specified directory.")
    else:
        print(f"\nFound {len(video_paths)} videos:")
        for v in video_paths:
            print(f"- {v}")

        # extract frames with motion-aware adaptive intervals
        print("\nStarting motion-aware frame extraction...")
        extract_diverse_frames(
            video_paths=video_paths,
            base_output_dir=OUTPUT_DIR,
            motion_threshold=MOTION_THRESHOLD
        )
        print("\nFrame extraction completed!")

Searching for videos in /content/drive/MyDrive/yolov11_test_cv/data/raw_video...

Found 6 videos:
- /content/drive/MyDrive/yolov11_test_cv/data/raw_video/1.MOV
- /content/drive/MyDrive/yolov11_test_cv/data/raw_video/2_1.MOV
- /content/drive/MyDrive/yolov11_test_cv/data/raw_video/3_1.MOV
- /content/drive/MyDrive/yolov11_test_cv/data/raw_video/3_2.MOV
- /content/drive/MyDrive/yolov11_test_cv/data/raw_video/4.MOV
- /content/drive/MyDrive/yolov11_test_cv/data/raw_video/4_1.MOV

Starting motion-aware frame extraction...

Processing 1 (6.0s, 179 frames)
Base interval: 0.96s (~28 frames)
Motion check every: 0.48s
Saved 30 diverse frames to /content/drive/MyDrive/yolov11_test_cv/data/extracted_frames
Frame selection diversity: 100.0% of target

Processing 2_1 (49.3s, 1478 frames)
Base interval: 1.80s (~54 frames)
Motion check every: 0.90s
Saved 35 diverse frames to /content/drive/MyDrive/yolov11_test_cv/data/extracted_frames
Frame selection diversity: 120.0% of target

Processing 3_1 (99.3s, 2

# Filter classes (due to labelImg)

In [ ]:
import os
from pathlib import Path

# format: {original_id: new_id}
class_id_mapping = {
    15: 0,  # container
    17: 1,  # cup
    24: 2,  # tea
    20: 3,  # soup
    21: 4,  # meat
    22: 5,  # salad
    23: 6,  # dish
    26: 7   # plate
}

folders = ['/content/drive/MyDrive/yolov11_test_cv/data/anno_labelimg']

for folder in folders:
    labels_path = Path(folder)

    for label_file in labels_path.glob('*.txt'):
        new_lines = []

        with open(label_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:  # skipping invalid lines
                continue

            original_id = int(parts[0])

            # check if this is a class I want to keep
            if original_id in class_id_mapping:
                new_id = class_id_mapping[original_id]
                new_line = f"{new_id} {' '.join(parts[1:])}\n"
                new_lines.append(new_line)

        with open(label_file, 'w') as f:  # overwrite original
            f.writelines(new_lines)

print("Class filtering and remapping complete!")

Class filtering and remapping complete!


# Prepare images to auto-labeling


In [ ]:
import os
import shutil
import yaml


BASE_PATH = '/content/drive/MyDrive/yolov11_test_cv/data'
ALL_IMAGES_DIR = os.path.join(BASE_PATH, 'extracted_frames')
ALL_LABELS_DIR = os.path.join(BASE_PATH, 'anno_labelimg')

YOLOV8_DIR = os.path.join(BASE_PATH, 'yolov8')
IMAGES_TRAIN = os.path.join(YOLOV8_DIR, 'images/train')
IMAGES_UNLABELED = os.path.join(YOLOV8_DIR, 'images/unlabeled')
LABELS_TRAIN = os.path.join(YOLOV8_DIR, 'labels/train')
CLASSES_FILE = os.path.join(YOLOV8_DIR, 'classes.txt')
DATA_YAML_FILE = os.path.join(YOLOV8_DIR, 'data.yaml')

os.makedirs(IMAGES_TRAIN, exist_ok=True)
os.makedirs(IMAGES_UNLABELED, exist_ok=True)
os.makedirs(LABELS_TRAIN, exist_ok=True)

# collecting base names of labeled images
labeled_image_names = set()
for label_file in os.listdir(ALL_LABELS_DIR):
    if label_file.endswith('.txt'):
        labeled_image_names.add(os.path.splitext(label_file)[0])

# images to train or unlabeled
for image_file in os.listdir(ALL_IMAGES_DIR):
    base_name, ext = os.path.splitext(image_file)
    src = os.path.join(ALL_IMAGES_DIR, image_file)

    if base_name in labeled_image_names:
        dst = os.path.join(IMAGES_TRAIN, image_file)
    else:
        dst = os.path.join(IMAGES_UNLABELED, image_file)

    shutil.copy2(src, dst)

# labels
for label_file in os.listdir(ALL_LABELS_DIR):
    if label_file.endswith('.txt'):
        shutil.copy2(
            os.path.join(ALL_LABELS_DIR, label_file),
            os.path.join(LABELS_TRAIN, label_file)
        )

# classes
default_classes = ['container', 'cup', 'tea', 'soup', 'meat', 'salad', 'dish', 'plate']
with open(CLASSES_FILE, 'w') as f:
    f.writelines(f"{cls}\n" for cls in default_classes)


data_yaml = {
    'path': YOLOV8_DIR,
    'train': 'images/train',
    'val': 'images/train',
    'names': {i: name for i, name in enumerate(default_classes)}
}
with open(DATA_YAML_FILE, 'w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print(f"YOLOv8 dataset prepared at: {YOLOV8_DIR}")
print(f"Training images: {IMAGES_TRAIN}")
print(f"Unlabeled images: {IMAGES_UNLABELED}")
print(f"data.yaml created: {DATA_YAML_FILE}")

In [ ]:
folder_path = "/content/drive/MyDrive/yolov11_test_cv/data/yolov8/images/unlabeled"

txt_files = [f for f in os.listdir(folder_path)]
print(f"Number of .txt files: {len(txt_files)}")

Number of .txt files: 499


# YOLOv8 semi-auto-labeling


In [ ]:
#!pip install ultralytics
#!pip uninstall -y torch torchvision torchaudio ultralytics
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
#!pip install ultralytics

In [ ]:
import os
import shutil
from ultralytics import YOLO
from tqdm import tqdm

BASE_PATH = '/content/drive/MyDrive/yolov11_test_cv/data/yolov8'
DATA_YAML = os.path.join(BASE_PATH, 'data.yaml')
UNLABELED_IMAGES = os.path.join(BASE_PATH, 'images', 'unlabeled')
UNLABELED_LABELS_OUT = os.path.join(BASE_PATH, 'labels', 'unlabeled')
os.makedirs(UNLABELED_LABELS_OUT, exist_ok=True)

# train on labeled images
model = YOLO('yolov8n.pt')
model.train(
    data=DATA_YAML,
    epochs=2,
    imgsz=640,
    batch=16,
    project=os.path.join(BASE_PATH, 'runs'),
    name='yolo_train'
)

# loading trained model
trained_model = YOLO(os.path.join(BASE_PATH, 'runs', 'yolo_train', 'weights', 'best.pt'))

# predict in batches
image_list = sorted([
    os.path.join(UNLABELED_IMAGES, fname)
    for fname in os.listdir(UNLABELED_IMAGES)
    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
])

batch_size = 10
print(f"Total images: {len(image_list)} | Batch size: {batch_size}")

for i in tqdm(range(0, len(image_list), batch_size)):
    batch = image_list[i:i+batch_size]
    trained_model.predict(
        source=batch,
        save_txt=True,
        save_conf=True,
        project=os.path.join(BASE_PATH, 'runs'),
        name='autolabel',
        conf=0.3,
        verbose=False
    )

# copy predictions
predicted_labels_dir = os.path.join(BASE_PATH, 'runs', 'autolabel', 'labels')

for fname in os.listdir(predicted_labels_dir):
    if fname.endswith('.txt'):
        shutil.copy(
            os.path.join(predicted_labels_dir, fname),
            os.path.join(UNLABELED_LABELS_OUT, fname)
        )

print(f"Auto-labeling complete! Labels saved in: {UNLABELED_LABELS_OUT}")


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.7.1+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolov11_test_cv/data/yolov8/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

train: Scanning /content/drive/MyDrive/yolov11_test_cv/data/yolov8/labels/train.cache... 180 images, 1 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.7±0.5 ms, read: 206.8±85.4 MB/s, size: 2062.3 KB)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning /content/drive/MyDrive/yolov11_test_cv/data/yolov8/labels/train.cache... 180 images, 1 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to /content/drive/MyDrive/yolov11_test_cv/data/yolov8/runs/yolo_train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /content/drive/MyDrive/yolov11_test_cv/data/yolov8/runs/yolo_train4
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

Results saved to /content/drive/MyDrive/yolov11_test_cv/data/yolov8/runs/autolabel51
10 labels saved to /content/drive/MyDrive/yolov11_test_cv/data/yolov8/runs/autolabel51/labels


 98%|█████████▊| 49/50 [04:31<00:05,  5.65s/it]

Results saved to /content/drive/MyDrive/yolov11_test_cv/data/yolov8/runs/autolabel52
9 labels saved to /content/drive/MyDrive/yolov11_test_cv/data/yolov8/runs/autolabel52/labels


100%|██████████| 50/50 [04:36<00:00,  5.53s/it]


✅ Auto-labeling complete! Labels saved in: /content/drive/MyDrive/yolov11_test_cv/data/yolov8/labels/unlabeled


In [ ]:
import os
import shutil

# base directory where autolabel folders exist
BASE_RUNS_PATH = '/content/drive/MyDrive/yolov11_test_cv/data/yolov8/runs'
TARGET_LABELS_PATH = '/content/drive/MyDrive/yolov11_test_cv/data/yolov8/labels_500_auto'

# check existence
os.makedirs(TARGET_LABELS_PATH, exist_ok=True)

# through all subfolders that start with 'autolabel'
for folder_name in os.listdir(BASE_RUNS_PATH):
    if folder_name.startswith('autolabel'):
        labels_path = os.path.join(BASE_RUNS_PATH, folder_name, 'labels')

        if os.path.isdir(labels_path):
            for label_file in os.listdir(labels_path):
                if label_file.endswith('.txt'):
                    src = os.path.join(labels_path, label_file)
                    dst = os.path.join(TARGET_LABELS_PATH, label_file)
                    shutil.copy2(src, dst)

print(f"All label files copied to: {TARGET_LABELS_PATH}")


In [ ]:
folder_path = '/content/drive/MyDrive/yolov11_test_cv/data/yolov8/labels_500_auto'

txt_files = [f for f in os.listdir(folder_path)]
print(f"Number of .txt files: {len(txt_files)}")


Number of .txt files: 499


# Train Test Val split

In [1]:
import os
import shutil
import yaml
import random
from tqdm import tqdm

class YOLOv11DatasetSplitter:
    def __init__(self, image_dir, label_manual_dir, label_auto_dir, classes_file, output_dir):
        self.image_dir = image_dir
        self.label_manual_dir = label_manual_dir
        self.label_auto_dir = label_auto_dir
        self.classes_file = classes_file
        self.output_dir = output_dir

        self.splits = ['train', 'val', 'test']
        for split in self.splits:
            os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
            os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)

        with open(classes_file, 'r') as f:
            self.class_names = [line.strip() for line in f.readlines()]

        # mapping of all images → label path: manual or auto
        self.label_map = self._map_labels()

    def _map_labels(self):
        label_map = {}
        for label_dir in [self.label_manual_dir, self.label_auto_dir]:
            for file in os.listdir(label_dir):
                if file.endswith('.txt'):
                    key = os.path.splitext(file)[0]
                    label_map[key] = os.path.join(label_dir, file)
        return label_map

    def _get_all_labeled_images(self):
        """Return only images that have a corresponding label file"""
        return [
            fname for fname in os.listdir(self.image_dir)
            if os.path.splitext(fname)[0] in self.label_map
        ]

    def _get_manual_images(self):
        return [
            fname for fname in os.listdir(self.image_dir)
            if os.path.exists(os.path.join(self.label_manual_dir, os.path.splitext(fname)[0] + '.txt'))
        ]

    def split_dataset(self, train_ratio=0.7, val_ratio=0.2):
        manual_images = self._get_manual_images()
        all_labeled_images = self._get_all_labeled_images()

        # test set from MANUALLY labeled images!!!
        random.seed(42)
        random.shuffle(manual_images)
        test_size = int(0.3 * len(manual_images))
        test_images = manual_images[:test_size]

        # train and val
        trainval_images = list(set(all_labeled_images) - set(test_images))
        random.shuffle(trainval_images)

        train_size = int(train_ratio * len(trainval_images))
        val_size = int(val_ratio * len(trainval_images))

        train_images = trainval_images[:train_size]
        val_images = trainval_images[train_size:train_size + val_size]

        print(f"Train: {len(train_images)} | Val: {len(val_images)} | Test: {len(test_images)}")

        self._copy_split(train_images, 'train')
        self._copy_split(val_images, 'val')
        self._copy_split(test_images, 'test')

        self._save_classes_txt()
        self._save_dataset_yaml()

    def _copy_split(self, images, split):
        for img_name in tqdm(images, desc=f"Copying {split}"):
            base = os.path.splitext(img_name)[0]
            img_src = os.path.join(self.image_dir, img_name)
            label_src = self.label_map.get(base)

            if not os.path.exists(img_src) or not label_src:
                continue

            img_dst = os.path.join(self.output_dir, 'images', split, img_name)
            label_dst = os.path.join(self.output_dir, 'labels', split, base + '.txt')

            shutil.copy2(img_src, img_dst)
            shutil.copy2(label_src, label_dst)

    def _save_classes_txt(self):
        out_path = os.path.join(self.output_dir, 'classes.txt')
        with open(out_path, 'w') as f:
            for cls in self.class_names:
                f.write(cls + '\n')

    def _save_dataset_yaml(self):
        yaml_data = {
            'train': os.path.join(self.output_dir, 'images', 'train'),
            'val': os.path.join(self.output_dir, 'images', 'val'),
            'test': os.path.join(self.output_dir, 'images', 'test'),
            'nc': len(self.class_names),
            'names': self.class_names
        }
        with open(os.path.join(self.output_dir, 'dataset.yaml'), 'w') as f:
            yaml.dump(yaml_data, f, sort_keys=False)


if __name__ == "__main__":
    image_dir = "/content/drive/MyDrive/yolov11_test_cv/data/extracted_frames"
    label_manual_dir = "/content/drive/MyDrive/yolov11_test_cv/data/anno_labelimg"
    label_auto_dir = "/content/drive/MyDrive/yolov11_test_cv/data/yolov8/labels_500_auto"
    classes_file = "/content/drive/MyDrive/yolov11_test_cv/data/classes.txt"
    output_dir = "/content/drive/MyDrive/yolov11_test_cv/yolov11_dataset"

    splitter = YOLOv11DatasetSplitter(
        image_dir=image_dir,
        label_manual_dir=label_manual_dir,
        label_auto_dir=label_auto_dir,
        classes_file=classes_file,
        output_dir=output_dir
    )

    splitter.split_dataset()
    print(f"\n Dataset prepared at: {output_dir}")
    print(f"Structure: images/train, val, test and labels/train, val, test")


Train: 437 | Val: 125 | Test: 54


Copying test: 100%|██████████| 54/54 [00:03<00:00, 17.94it/s]


 Dataset prepared at: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset
Structure: images/train, val, test and labels/train, val, test


In [2]:
def check_yolo_dataset(base_path):
    splits = ['train', 'val', 'test']
    stats = {}

    for split in splits:
        print(f"\n Checking {split} set...")
        img_dir = os.path.join(base_path, 'images', split)
        label_dir = os.path.join(base_path, 'labels', split)

        img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
        label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

        img_bases = set(os.path.splitext(f)[0] for f in img_files)
        label_bases = set(os.path.splitext(f)[0] for f in label_files)

        # Checking for missing labels or images
        missing_labels = img_bases - label_bases
        missing_images = label_bases - img_bases

        # checking for empty label files
        empty_labels = []
        bad_format_labels = []
        for file in label_files:
            path = os.path.join(label_dir, file)
            with open(path, 'r') as f:
                lines = f.readlines()
                if len(lines) == 0:
                    empty_labels.append(file)
                else:
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) < 5:
                            bad_format_labels.append(file)
                            break

        stats[split] = {
            'images': len(img_files),
            'labels': len(label_files),
            'missing_labels': list(missing_labels),
            'missing_images': list(missing_images),
            'empty_labels': empty_labels,
            'bad_format_labels': bad_format_labels
        }

        print(f"Images: {len(img_files)} | Labels: {len(label_files)}")
        print(f"Missing labels: {len(missing_labels)}")
        print(f"Missing images: {len(missing_images)}")
        print(f"Empty label files: {len(empty_labels)}")
        print(f"Bad format labels: {len(bad_format_labels)}")

    return stats

if __name__ == "__main__":
    dataset_path = "/content/drive/MyDrive/yolov11_test_cv/yolov11_dataset"
    summary = check_yolo_dataset(dataset_path)



 Checking train set...
Images: 437 | Labels: 437
Missing labels: 0
Missing images: 0
Empty label files: 1
Bad format labels: 0

 Checking val set...
Images: 125 | Labels: 125
Missing labels: 0
Missing images: 0
Empty label files: 0
Bad format labels: 0

 Checking test set...
Images: 54 | Labels: 54
Missing labels: 0
Missing images: 0
Empty label files: 0
Bad format labels: 0


# Augmentation

In [4]:
import os
import cv2
import yaml
import numpy as np
import albumentations as A
from tqdm import tqdm
from pathlib import Path

class YOLODatasetAugmentor:
    def __init__(self, input_image_dir, input_label_dir, output_dir, augmentations_per_image=2, yaml_path='dataset.yaml'):
        self.image_dir = self._validate_directory(input_image_dir, "Input image")
        self.label_dir = self._validate_directory(input_label_dir, "Input label")
        self.output_dir = output_dir
        self.augmentations_per_image = max(1, int(augmentations_per_image))
        self.yaml_path = self._validate_yaml_path(yaml_path)

        # output directories
        self.aug_image_dir = os.path.join(output_dir, 'images', 'train')
        self.aug_label_dir = os.path.join(output_dir, 'labels', 'train')
        os.makedirs(self.aug_image_dir, exist_ok=True)
        os.makedirs(self.aug_label_dir, exist_ok=True)

        # init yaml config
        self.yaml_data = self._initialize_yaml_data()

        # albumentations transform
        self.transform = self._create_augmentation_pipeline()

    def _validate_directory(self, path, dir_type):
        if not os.path.exists(path):
            raise ValueError(f"{dir_type} directory does not exist: {path}")
        if not os.access(path, os.R_OK):
            raise ValueError(f"{dir_type} directory is not readable: {path}")
        return path

    def _validate_yaml_path(self, yaml_path):
        yaml_dir = os.path.dirname(yaml_path)
        if yaml_dir and not os.path.exists(yaml_dir):
            raise ValueError(f"YAML directory does not exist: {yaml_dir}")
        if os.path.exists(yaml_path) and not os.access(yaml_path, os.R_OK):
            raise ValueError(f"YAML file is not readable: {yaml_path}")
        return yaml_path

    def _load_class_names(self):
        classes_file = os.path.join(os.path.dirname(self.yaml_path), 'classes.txt')
        if os.path.exists(classes_file):
            with open(classes_file, 'r') as f:
                return [line.strip() for line in f if line.strip()]
        print("Warning: classes.txt not found. Using default names.")
        return [f"class_{i}" for i in range(8)]

    def _initialize_yaml_data(self):
        class_names = self._load_class_names()
        return {
            'train': 'images/train',
            'val': 'images/val',
            'test': 'images/test',
            'nc': len(class_names),
            'names': class_names
        }

    def _create_augmentation_pipeline(self):
        return A.Compose([
            A.RandomResizedCrop(size=(640, 640), scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.Affine(scale=(0.9, 1.1), rotate=(-15, 15), translate_percent=(0.05, 0.05), p=0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
            A.RandomGamma(gamma_limit=(80, 120), p=0.3),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            A.GaussianBlur(blur_limit=(3, 7), p=0.2),

            A.CoarseDropout(max_holes=8, max_h_size=32, max_w_size=32,
                min_holes=1, min_h_size=8, min_w_size=8, mask_fill_value=0, p=0.3),
        ], bbox_params=A.BboxParams(
            format='yolo',
            min_area=16,
            min_visibility=0.1,
            label_fields=['class_labels'],
            check_each_transform=False  # disables internal validation
        ))

    def _load_yolo_labels(self, label_path):
        if not os.path.exists(label_path):
            return [], []

        boxes, class_labels = [], []
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    try:
                        cls = int(float(parts[0]))
                        box = list(map(float, parts[1:5]))  # yolo format

                        # Validate box values have to be in [0, 1]
                        if any(b < 0 or b > 1 for b in box):
                            print(f"Invalid box in {label_path}: {box}")
                            continue

                        boxes.append(box)
                        class_labels.append(cls)
                    except Exception as e:
                        print(f"Label parse error in {label_path}: {e}")
                        continue

        return boxes, class_labels

    def _save_yolo_labels(self, label_path, boxes, class_labels):
        with open(label_path, 'w') as f:
            for cls, box in zip(class_labels, boxes):
                f.write(f"{cls} {' '.join(map(str, box))}\n")

    def _update_yaml(self):
        try:
            with open(self.yaml_path, 'w') as f:
                yaml.dump(self.yaml_data, f, sort_keys=False)
            print(f"Saved dataset.yaml to: {self.yaml_path}")
        except Exception as e:
            print(f"Error saving YAML: {e}")

    def augment_dataset(self):
        image_files = [f for f in os.listdir(self.image_dir)
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        if not image_files:
            print(f"No images found in {self.image_dir}")
            return

        for img_file in tqdm(image_files, desc="Augmenting"):
            base_name = os.path.splitext(img_file)[0]
            img_path = os.path.join(self.image_dir, img_file)
            label_path = os.path.join(self.label_dir, f"{base_name}.txt")

            image = cv2.imread(img_path)
            if image is None:
                print(f"Could not read image: {img_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            boxes, class_labels = self._load_yolo_labels(label_path)
            if not boxes:
                print(f"No labels found for {img_file}")
                continue

            # saving original image and labels
            aug_img_path = os.path.join(self.aug_image_dir, f"{base_name}_aug0.jpg")
            aug_lbl_path = os.path.join(self.aug_label_dir, f"{base_name}_aug0.txt")
            cv2.imwrite(aug_img_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
            self._save_yolo_labels(aug_lbl_path, boxes, class_labels)

            # generate augmentations
            for i in range(1, self.augmentations_per_image):
                try:
                    transformed = self.transform(image=image, bboxes=boxes, class_labels=class_labels)

                    clipped_bboxes = []
                    clipped_classes = []
                    for box, cls in zip(transformed['bboxes'], transformed['class_labels']):
                        x, y, w, h = box
                        x = min(max(x, 0.0), 1.0)
                        y = min(max(y, 0.0), 1.0)
                        w = min(max(w, 0.0), 1.0)
                        h = min(max(h, 0.0), 1.0)
                        if w > 0 and h > 0:
                            clipped_bboxes.append([x, y, w, h])
                            clipped_classes.append(cls)

                    if not clipped_bboxes:
                        continue

                    aug_img_path = os.path.join(self.aug_image_dir, f"{base_name}_aug{i}.jpg")
                    aug_lbl_path = os.path.join(self.aug_label_dir, f"{base_name}_aug{i}.txt")
                    cv2.imwrite(aug_img_path, cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR))
                    self._save_yolo_labels(aug_lbl_path, clipped_bboxes, clipped_classes)

                except Exception as e:
                    print(f"Augmentation error for {img_file} (aug{i}): {e}")
                    continue

        self._update_yaml()

if __name__ == "__main__":
    input_image_dir = "/content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/images/train"
    input_label_dir = "/content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/labels/train/"
    output_dir = "/content/drive/MyDrive/yolov11_test_cv/yolov11_dataset"
    yaml_path = os.path.join(output_dir, "dataset.yaml")

    augmentor = YOLODatasetAugmentor(
        input_image_dir=input_image_dir,
        input_label_dir=input_label_dir,
        output_dir=output_dir,
        augmentations_per_image=3,
        yaml_path=yaml_path
    )

    augmentor.augment_dataset()
    print("Augmentation complete.")


/tmp/ipython-input-4-3642194342.py:70: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
/tmp/ipython-input-4-3642194342.py:73: UserWarning: Argument(s) 'max_holes, max_h_size, max_w_size, min_holes, min_h_size, min_w_size, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_h_size=32, max_w_size=32,
Augmenting:   4%|▍         | 18/437 [00:12<02:42,  2.57it/s]

Augmentation error for 4_frame_001224.jpg (aug1): Expected y_min for bbox [ 3.4069550e-01 -4.9173832e-07  4.8841852e-01  3.0292648e-01
  7.0000000e+00] to be in the range [0.0, 1.0], got -4.917383193969727e-07.
Augmentation error for 4_frame_001224.jpg (aug2): Expected y_min for bbox [ 3.4069550e-01 -4.9173832e-07  4.8841852e-01  3.0292648e-01
  7.0000000e+00] to be in the range [0.0, 1.0], got -4.917383193969727e-07.


Augmenting:   7%|▋         | 31/437 [00:21<03:31,  1.92it/s]

Augmentation error for 4_frame_000225.jpg (aug1): Expected y_min for bbox [ 3.4031349e-01 -4.9173832e-07  4.8747849e-01  3.0182052e-01
  7.0000000e+00] to be in the range [0.0, 1.0], got -4.917383193969727e-07.
Augmentation error for 4_frame_000225.jpg (aug2): Expected y_min for bbox [ 3.4031349e-01 -4.9173832e-07  4.8747849e-01  3.0182052e-01
  7.0000000e+00] to be in the range [0.0, 1.0], got -4.917383193969727e-07.


Augmenting:   8%|▊         | 34/437 [00:22<02:28,  2.71it/s]

...

Augmentation error for 4_frame_000348.jpg (aug1): Expected y_min for bbox [ 3.4029150e-01 -5.0663948e-07  4.8786253e-01  3.0122450e-01
  7.0000000e+00] to be in the range [0.0, 1.0], got -5.066394805908203e-07.
Augmentation error for 4_frame_000348.jpg (aug2): Expected y_min for bbox [ 3.4029150e-01 -5.0663948e-07  4.8786253e-01  3.0122450e-01
  7.0000000e+00] to be in the range [0.0, 1.0], got -5.066394805908203e-07.


Augmenting:  99%|█████████▉| 434/437 [06:11<00:01,  1.81it/s]

Augmentation error for 4_frame_000275.jpg (aug1): Expected y_min for bbox [ 3.4032550e-01 -5.0663948e-07  4.8755452e-01  3.0189049e-01
  7.0000000e+00] to be in the range [0.0, 1.0], got -5.066394805908203e-07.
Augmentation error for 4_frame_000275.jpg (aug2): Expected y_min for bbox [ 3.4032550e-01 -5.0663948e-07  4.8755452e-01  3.0189049e-01
  7.0000000e+00] to be in the range [0.0, 1.0], got -5.066394805908203e-07.


Augmenting: 100%|██████████| 437/437 [06:16<00:00,  1.16it/s]

Saved dataset.yaml to: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/dataset.yaml
Augmentation complete.


# Train YOLOv11


In [2]:
import os
import yaml
import torch
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import ParameterGrid
from PIL import Image
import psutil

class YOLOv11Trainer:
    def __init__(self, dataset_yaml, base_yaml, train_yaml, inference_yaml):
        """Initialize trainer with configuration files"""
        # set base dataset path
        self.dataset_base_path = Path(dataset_yaml).parent
        print(f"Dataset base path: {self.dataset_base_path}")

        # system checks
        self._check_system_resources()

        # device configs
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Using device: {self.device}")

        # Load configs
        self.dataset_cfg = self._load_and_validate_yaml(dataset_yaml)
        self.base_cfg = self._load_and_validate_yaml(base_yaml)
        self.train_cfg = self._load_and_validate_yaml(train_yaml)
        self.inference_cfg = self._load_and_validate_yaml(inference_yaml)

        # validate dataset structure
        self._validate_dataset_structure()

        # optimize for available resources
        self._optimize_for_resources()

        # setup output directories
        self.output_dir = Path('runs') / 'train'
        self.output_dir.mkdir(parents=True, exist_ok=True)

        # init model
        self.model = self._init_model()
        self.history = []
        self.best_model_path = None

    def _check_system_resources(self):
        """Verify sufficient system resources"""
        mem = psutil.virtual_memory()
        print(f"Available RAM: {mem.available / (1024**3):.2f} GB")
        if mem.available < 4 * 1024**3:  # Less than 4GB
            raise RuntimeError("Insufficient RAM (minimum 4GB required)")

    def _load_and_validate_yaml(self, path):
        """Load and validate YAML configuration"""
        path = Path(path)
        if not path.exists():
            raise FileNotFoundError(f"Config file not found: {path}")
        with open(path) as f:
            config = yaml.safe_load(f)
        return config

    def _validate_dataset_structure(self):
        """Validate dataset folder structure and files"""
        required_dirs = [
            self.dataset_base_path / 'images' / 'train',
            self.dataset_base_path / 'images' / 'val',
            self.dataset_base_path / 'images' / 'test',
            self.dataset_base_path / 'labels' / 'train',
            self.dataset_base_path / 'labels' / 'val',
            self.dataset_base_path / 'labels' / 'test'
        ]
        for dir_path in required_dirs:
            if not dir_path.exists():
                raise FileNotFoundError(f"Required directory not found: {dir_path}")
        # check classes.txt optional, since dataset.yaml includes names
        classes_file = self.dataset_base_path / 'classes.txt'
        if classes_file.exists():
            with open(classes_file) as f:
                class_names = [line.strip() for line in f if line.strip()]
            if class_names != self.dataset_cfg['names']:
                print(f"Warning: classes.txt names {class_names} differ from dataset.yaml names {self.dataset_cfg['names']}")

    def _optimize_for_resources(self):
        """Adjust parameters based on available resources"""
        if self.device == 'cpu':
            print("Optimizing for CPU...")
            # reduce memory-intensive settings
            self.train_cfg['training']['batch_size'] = min(8, self.train_cfg['training']['batch_size'])
            self.base_cfg['system']['num_workers'] = min(2, self.base_cfg['system']['num_workers'])
            self.base_cfg['data']['img_size'] = [min(512, self.base_cfg['data']['img_size'][0])] * 2
            # disable heavy augmentations
            self.train_cfg['data_augmentation']['mosaic'] = 0.0
            self.train_cfg['data_augmentation']['mixup'] = 0.0

    def _init_model(self):
        """Initialize model with memory awareness"""
        model_size = self.base_cfg['model']['architecture'][-1]  # s/m/l/x I chosed s
        if self.device == 'cpu' and model_size in ['l', 'x']:
            print(f"Reducing model size from yolov11{model_size} to yolov11s for CPU")
            model_size = 's'

        model_name = f'yolov11{model_size}'

        try:
            if self.base_cfg['model']['pretrained']:
                try:
                    print(f"Loading {model_name}.pt")
                    return YOLO(f'{model_name}.pt').to(self.device)
                except:
                    print(f"Falling back to yolov8{model_size}.pt")
                    return YOLO(f'yolov8{model_size}.pt').to(self.device)
            print(f"Initializing from scratch: {model_name}.yaml")
            return YOLO(f'{model_name}.yaml').to(self.device)
        except Exception as e:
            raise RuntimeError(f"Model initialization failed: {e}")

    def train(self, params=None):
        """Train model with memory control"""
        try:
            # create temporary dataset yaml
            temp_yaml = self._create_temp_dataset_yaml()

            # dase configs
            cfg = {
                'data': str(temp_yaml),
                'epochs': self.train_cfg['training']['epochs'],
                'batch': self.train_cfg['training']['batch_size'],
                'imgsz': self.base_cfg['data']['img_size'][0],
                'device': self.device,
                'workers': self.base_cfg['system']['num_workers'],
                'seed': self.base_cfg['system']['seed'],
                'project': str(self.output_dir),
                'name': self.base_cfg['logging']['name'],
                'exist_ok': True,
                'cos_lr': self.train_cfg['scheduler']['type'] == 'cosine',
                'lr0': self.train_cfg['optimizer']['lr'],
                'momentum': self.train_cfg['optimizer']['momentum'],
                'weight_decay': self.train_cfg['optimizer']['weight_decay'],
                'fliplr': self.train_cfg['data_augmentation']['fliplr'],
            }

            # apply custom parameters if provided
            if params:
                cfg.update(params)
                cfg['name'] = f"{cfg['name']}_opt_{len(self.history)+1}"

            print("\nTraining configuration:")
            for k, v in cfg.items():
                print(f"{k}: {v}")

            # clearing memory before training
            torch.cuda.empty_cache() if self.device == 'cuda' else None

            # rain with memory monitoring
            results = self._train_with_memory_control(cfg)

            # save results and model path
            self.history.append({'params': cfg, 'results': results})
            self.best_model_path = Path(cfg['project']) / cfg['name'] / 'weights' / 'best.pt'

            # clean up
            temp_yaml.unlink(missing_ok=True)

            return results

        except Exception as e:
            print(f"Training error: {e}")
            raise

    def _create_temp_dataset_yaml(self):
        """Create temporary dataset YAML with absolute paths"""
        temp_yaml_path = self.output_dir / 'temp_dataset.yaml'
        with open(temp_yaml_path, 'w') as f:
            yaml.dump({
                'train': str(self.dataset_base_path / self.dataset_cfg['train']),
                'val': str(self.dataset_base_path / self.dataset_cfg['val']),
                'test': str(self.dataset_base_path / self.dataset_cfg['test']),
                'nc': self.dataset_cfg['nc'],
                'names': self.dataset_cfg['names']
            }, f)
        return temp_yaml_path

    def _train_with_memory_control(self, cfg):
        """Handle memory issues during training"""
        try:
            return self.model.train(**cfg)
        except RuntimeError as e:
            if 'CUDA out of memory' in str(e):
                print("\nOut of memory! Reducing batch size...")
                cfg['batch'] = max(1, cfg['batch'] // 2)
                print(f"New batch size: {cfg['batch']}")
                return self.model.train(**cfg)
            raise

    def optimize_hyperparameters(self, n_iter=2):
        """Run hyperparameter optimization"""
        param_grid = {
            'lr0': [0.01, 0.001],
            'batch': [8, 4] if self.device == 'cpu' else [16, 32],
            'weight_decay': [0.0005, 0.001]
        }

        best_score = 0
        best_params = None

        print(f"\nStarting hyperparameter optimization ({n_iter} iterations)...")

        for i, params in enumerate(ParameterGrid(param_grid)):
            if i >= n_iter:
                break

            print(f"\nIteration {i+1}/{n_iter}: {params}")

            try:
                self.train(params)
                metrics = self.validate()

                if metrics and metrics['mAP50-95'] > best_score:
                    best_score = metrics['mAP50-95']
                    best_params = params

                print(f"Current mAP50-95: {metrics['mAP50-95']:.4f}")

            except Exception as e:
                print(f"Iteration {i+1} failed: {e}")
                continue

        print(f"\nBest mAP50-95: {best_score:.4f}")
        print(f"Best parameters: {best_params}")

        return best_params, best_score

    def validate(self):
        """Validate model on validation set"""
        try:
            if not self.best_model_path:
                raise ValueError("No trained model available for validation")

            print("\nRunning validation...")
            model = YOLO(str(self.best_model_path))

            metrics = model.val(
                data=str(self._create_temp_dataset_yaml()),
                batch=self.train_cfg['training']['batch_size'],
                imgsz=self.base_cfg['data']['img_size'][0],
                device=self.device
            )

            val_results = {
                'mAP50': metrics.box.map50,
                'mAP50-95': metrics.box.map,
                'precision': metrics.box.p,
                'recall': metrics.box.r,
                'f1': metrics.box.f1
            }

            print("\nValidation Results:")
            for k, v in val_results.items():
                print(f"{k}: {v:.4f}")

            return val_results

        except Exception as e:
            print(f"Validation error: {e}")
            return None

    def test(self, num_samples=5, save_dir='test_results'):
        """Test model on test set with visualization"""
        try:
            if not self.best_model_path:
                raise ValueError("No trained model available for testing")

            print(f"\nTesting model on {num_samples} samples...")
            test_dir = self.output_dir / save_dir
            test_dir.mkdir(exist_ok=True)

            model = YOLO(str(self.best_model_path))

            # Get test images
            test_images = list((self.dataset_base_path / self.dataset_cfg['test']).glob('*.[jp][pn]g'))
            if not test_images:
                raise FileNotFoundError(f"No images found in test directory: {self.dataset_base_path / self.dataset_cfg['test']}")

            # Process sample images
            sample_images = test_images[:min(num_samples, len(test_images))]
            test_results = []

            for img_path in sample_images:
                results = model.predict(
                    source=str(img_path),
                    imgsz=self.base_cfg['data']['img_size'][0],
                    conf=self.inference_cfg['inference']['conf_thres'],
                    iou=self.inference_cfg['inference']['iou_thres'],
                    device=self.device
                )

                # Save visualization
                for r in results:
                    im_array = r.plot()
                    im = Image.fromarray(im_array[..., ::-1])
                    save_path = test_dir / f"result_{img_path.name}"
                    im.save(save_path)
                    print(f"Saved: {save_path}")
                    test_results.append({
                        'image': img_path.name,
                        'predictions': len(r.boxes),
                        'average_conf': r.boxes.conf.mean().item() if len(r.boxes) > 0 else 0
                    })

            # Full test set evaluation
            test_metrics = model.val(
                data=str(self._create_temp_dataset_yaml()),
                split='test',
                batch=self.train_cfg['training']['batch_size'],
                imgsz=self.base_cfg['data']['img_size'][0],
                device=self.device
            )

            full_test_results = {
                'mAP50': test_metrics.box.map50,
                'mAP50-95': test_metrics.box.map,
                'precision': test_metrics.box.p,
                'recall': test_metrics.box.r,
                'f1': test_metrics.box.f1,
                'sample_results': test_results
            }

            print("\nTest Set Metrics:")
            for k, v in full_test_results.items():
                if k != 'sample_results':
                    print(f"{k}: {v:.4f}")

            # Save test results
            self._save_test_results(full_test_results, test_dir)

            return full_test_results

        except Exception as e:
            print(f"Testing error: {e}")
            return None

    def _save_test_results(self, results, test_dir):
        """Save test results and update report"""
        try:
            # Save numerical results
            df = pd.DataFrame([{k:v for k,v in results.items() if k != 'sample_candidates'}])
            df.to_csv(test_dir / 'test_metrics.csv', index=False)

            # Save sample predictions
            sample_df = pd.DataFrame(results['sample_results'])
            sample_df.to_csv(test_dir / 'sample_predictions.csv', index=False)

            # Update report
            self._update_report_with_test_results(results, test_dir)

        except Exception as e:
            print(f"Error saving test results: {e}")

    def _update_report_with_test_results(self, results, test_dir):
        """Add test results to the report"""
        try:
            report_path = self.output_dir / 'training_report.md'

            with open(report_path, 'a') as f:
                f.write("\n## Test Set Evaluation\n")
                f.write(f"- mAP50: {results['mAP50']:.4f}\n")
                f.write(f"- mAP50-95: {results['mAP50-95']:.4f}\n")
                f.write(f"- Precision: {results['precision']:.4f}\n")
                f.write(f"- Recall: {results['recall']:.4f}\n")
                f.write(f"- F1 Score: {results['f1']:.4f}\n\n")

                f.write("### Sample Predictions\n")
                for sample in results['sample_results'][:3]:  # Show first 3 samples
                    img_name = sample['image']
                    f.write(f"- {img_name}: {sample['predictions']} detections (avg conf: {sample['average_conf']:.2f})\n")
                    img_path = test_dir / f"result_{img_name}"
                    if img_path.exists():
                        f.write(f"![{img_name}]({test_dir.name}/result_{img_name})\n")

            print(f"Updated report with test results: {report_path}")

        except Exception as e:
            print(f"Error updating report: {e}")

    def generate_report(self):
        """Generate comprehensive training report"""
        try:
            if not self.history:
                raise ValueError("No training history to report")

            report_path = self.output_dir / 'training_report.md'

            with open(report_path, 'w') as f:
                f.write("# YOLOv11 Training Report\n\n")

                # System and dataset info
                f.write("## System Information\n")
                f.write(f"- Device: {self.device}\n")
                f.write(f"- CUDA Available: {torch.cuda.is_available()}\n")
                if torch.cuda.is_available():
                    f.write(f"- CUDA Device: {torch.cuda.get_device_name()}\n")
                f.write("\n")

                f.write("## Dataset Information\n")
                f.write(f"- Classes: {self.dataset_cfg['names']}\n")
                f.write(f"- Training samples: {len(list((self.dataset_base_path / self.dataset_cfg['train']).glob('*.[jp][pn]g')))}\n")
                f.write(f"- Validation samples: {len(list((self.dataset_base_path / self.dataset_cfg['val']).glob('*.[jp][pn]g')))}\n")
                f.write(f"- Test samples: {len(list((self.dataset_base_path / self.dataset_cfg['test']).glob('*.[jp][pn]g')))}\n\n")

                # Training summary
                f.write("## Training Summary\n")
                for i, run in enumerate(self.history):
                    f.write(f"### Run {i+1}\n")
                    f.write(f"- Epochs: {run['params']['epochs']}\n")
                    f.write(f"- Batch size: {run['params']['batch']}\n")
                    f.write(f"- Learning rate: {run['params']['lr0']}\n")
                    f.write(f"- Image size: {run['params']['imgsz']}\n\n")

                # Validation results
                val_results = self.validate()
                if val_results:
                    f.write("## Validation Results\n")
                    for k, v in val_results.items():
                        f.write(f"- {k}: {v:.4f}\n")
                    f.write("\n")

            print(f"Report generated: {report_path}")
            return report_path

        except Exception as e:
            print(f"Report generation error: {e}")
            return None


In [5]:
if __name__ == "__main__":
    try:
        # Configuration paths
        config_files = {
            'dataset_yaml': "/content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/dataset.yaml",
            'base_yaml': "/content/drive/MyDrive/yolov11_test_cv/configs/base.yml",
            'train_yaml': "/content/drive/MyDrive/yolov11_test_cv/configs/train.yml",
            'inference_yaml': "/content/drive/MyDrive/yolov11_test_cv/configs/inference.yml"
        }

        # Debug: List files in configs directory
        configs_dir = "/content/drive/MyDrive/yolov11_test_cv/configs"
        print(f"Checking configs directory: {configs_dir}")
        if os.path.exists(configs_dir):
            print(f"Files found: {os.listdir(configs_dir)}")
        else:
            print(f"Configs directory does not exist: {configs_dir}")

        # Verify config files exist
        for name, path in config_files.items():
            if not Path(path).exists():
                raise FileNotFoundError(f"Config file not found: {path}")

        # Initialize trainer
        trainer = YOLOv11Trainer(**config_files)

        # Training pipeline
        print("\n1. Starting training...")
        trainer.train()

        print("\n2. Optimizing hyperparameters...")
        trainer.optimize_hyperparameters(n_iter=2)

        print("\n3. Generating report...")
        trainer.generate_report()

        print("\n4. Running validation...")
        trainer.validate()

        print("\n5. Testing and visualizing results...")
        trainer.test(num_samples=5)

        print(f"\nTraining complete! Results saved in: {trainer.output_dir}")

    except Exception as e:
        print(f"\nCritical error: {e}")

Checking configs directory: /content/drive/MyDrive/yolov11_test_cv/configs
Files found: ['inference.yml', 'base.yml', 'train.yml']
Dataset base path: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset
Available RAM: 10.45 GB
Using device: cpu
Optimizing for CPU...
Loading yolov11s.pt
Falling back to yolov8s.pt

1. Starting training...

Training configuration:
data: runs/train/temp_dataset.yaml
epochs: 2
batch: 8
imgsz: 512
device: cpu
workers: 2
seed: 0
project: runs/train
name: yolov11-exp
exist_ok: True
cos_lr: True
lr0: 0.01
momentum: 0.937
weight_decay: 0.0005
fliplr: 0.5
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.7.1+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=runs/train/temp_dataset.yaml, degrees=0.0, deterministic=True, device=cpu, df

train: Scanning /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/labels/train... 1695 images, 1 backgrounds, 355 corrupt: 100%|██████████| 1695/1695 [00:50<00:00, 33.29it/s]

train: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/images/train/1_frame_000009.jpg: ignoring corrupt image/label: labels require 5 columns, 6 columns detected
train: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/images/train/1_frame_000014.jpg: ignoring corrupt image/label: labels require 5 columns, 6 columns detected
train: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/images/train/1_frame_000020.jpg: ignoring corrupt image/label: labels require 5 columns, 6 columns detected
train: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/images/train/1_frame_000026.jpg: ignoring corrupt image/label: labels require 5 columns, 6 columns detected
train: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/images/train/1_frame_000028.jpg: ignoring corrupt image/label: labels require 5 columns, 6 columns detected
train: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/images/train/1_frame_000050.jpg: ignoring corrupt image/label: labels require 5 columns,

train: New cache created: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

                   all         32        202      0.995      0.991      0.992      0.861
                   cup         32         87          1      0.978      0.995      0.829
                  meat         15         15      0.991          1      0.995      0.852
                 salad         15         30      0.996          1      0.995      0.831
                  dish         27         27      0.993          1      0.995       0.91
                 plate         14         43      0.997      0.977       0.98      0.883
Speed: 0.7ms preprocess, 264.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val4

Validation Results:
mAP50: 0.9920
mAP50-95: 0.8612
Validation error: unsupported format string passed to numpy.ndarray.__format__

5. Testing and visualizing results...

Testing model on 5 samples...

image 1/1 /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/images/test/4_1_frame_000135.jpg: 288x512 3 cups, 1 dish, 2 plates, 228.7ms
Speed

val: Scanning /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/labels/test... 54 images, 0 backgrounds, 0 corrupt: 100%|██████████| 54/54 [00:00<00:00, 64.32it/s]


val: New cache created: /content/drive/MyDrive/yolov11_test_cv/yolov11_dataset/labels/test.cache


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:17<00:00,  2.50s/it]


                   all         54        337      0.995      0.984      0.991      0.845
                   cup         54        150          1       0.99      0.995       0.83
                  meat         22         22      0.993          1      0.995        0.8
                 salad         22         44      0.997          1      0.995      0.822
                  dish         49         49      0.989      0.959      0.992      0.907
                 plate         29         72      0.997      0.972      0.979      0.865
Speed: 0.7ms preprocess, 238.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val5

Test Set Metrics:
mAP50: 0.9912
mAP50-95: 0.8447
Testing error: unsupported format string passed to numpy.ndarray.__format__

Training complete! Results saved in: runs/train


In [6]:
from google.colab import files
import shutil

# Zip the folder
shutil.make_archive('/content/runs', 'zip', '/content/runs')

# Download the zipped folder
files.download('/content/runs.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>